<a href="https://colab.research.google.com/github/ravi02512/Sentiment-classification-using-LSTM-Network/blob/master/sentiment%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)



import pandas as pd 
import numpy as np

Using TensorFlow backend.


In [0]:
#helper functions


import numpy as np
import pandas as pd

def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map


def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

    

In [0]:
train = pd.read_csv('/content/drive/My Drive/sentiment/train_2kmZucJ.csv')
test= pd.read_csv('/content/drive/My Drive/sentiment/test_oJQbWVk.csv')

In [0]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('/content/drive/My Drive/sentiment/glove.6B/glove.6B/glove.6B.300d.txt')

dictionary=word_to_vec_map.keys()

In [0]:
import re
clean_train=[]
for i in range(len(train)):
  new=re.sub('[^A-Za-z ]+', ' ', train['tweet'][i])
  clean_train.append(new)

In [110]:
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
def clean_sentence(sentence):
  tokens=[]
  word_list=sentence.split(' ')
  for w in word_list:
    if w in dictionary:
#       if w not in stopwords:
      if set(w) & set('aeiou')!=set():
        tokens.append(w)

  return ' '.join(tokens)

In [0]:
final_train=[]
for i in range(len(clean_train)):
  final_train.append(clean_sentence(clean_train[i]))
  

In [176]:
final_train[1]

'a silicon case to uncle yay instagram com'

In [0]:
tokens=[]
for i in range(len(clean_train)):
  tokens.extend(clean_train[i].split(' '))

In [0]:
count_df=pd.DataFrame(tokens,columns=['words'])
count_df=count_df['words'].value_counts().reset_index().rename(columns={'index':'words', 'words':'counts'})
count_df['word length']=[len(w) for w in count_df['words']]
count_df.head()

,words,counts,word length
0,,75832,0
1,http,3442,4
2,com,3389,3
3,iphone,2798,6
4,p,2724,1


In [0]:
g=[]
for w in count_df['words']:
  if w in dictionary:
    g.append(w)
    
print('number of words belong to dictionary'len(g))

SyntaxError: ignored

In [0]:
length=[]
for i in range(len(final_train)):
  length.append(len([word.lower() for word in final_train[i].split()]))

In [178]:
max_len=np.max(length)
print(max_len)

49


In [0]:

def sentences_to_indices(X, word_to_index, max_len):
    """
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]                        
    
    X_indices = np.zeros((m,max_len))
    
    for i in range(m):                                     
        sentence_words = [word.lower() for word in X[i].split()]
        j = 0
        for w in sentence_words:
          if w.lower() in dictionary:
              X_indices[i, j] = word_to_index[w.lower()]
              #
              j = j + 1


    
    return X_indices

1193515

In [0]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                 
    emb_dim = word_to_vec_map["cucumber"].shape[0]      
    
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    
    for word, index in word_to_index.items():
      if index!= 1800:
        emb_matrix[index, :] = word_to_vec_map[word]

    embedding_layer = Embedding(vocab_len, emb_dim, trainable = False)
    

    embedding_layer.build((None,))
    
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [0]:

def my_model(input_shape, word_to_vec_map, word_to_index):
    """
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    sentence_indices = Input(input_shape, dtype = 'int32')
    
  
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    embeddings = embedding_layer(sentence_indices)
    
    X = LSTM(128, return_sequences = True)(embeddings)
    X = Dropout(0.5)(X)
    
    X = LSTM(128, return_sequences=False)(X)
    X = Dropout(0.5)(X)
    
    X = Dense(2)(X)
    X = Activation('softmax')(X)
    
    model = Model(inputs = sentence_indices, outputs = X)
    
    
    return model

In [198]:
import gc 
gc.collect()

439

In [199]:
model = my_model((max_len,), word_to_vec_map, word_to_index)
model.summary()

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 49)                0         
_________________________________________________________________
embedding_13 (Embedding)     (None, 49, 300)           120000300 
_________________________________________________________________
lstm_25 (LSTM)               (None, 49, 128)           219648    
_________________________________________________________________
dropout_23 (Dropout)         (None, 49, 128)           0         
_________________________________________________________________
lstm_26 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_24 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 2)                 258

In [0]:
from keras import optimizers
# sgd = optimizers.SGD(lr=0.1, decay=1e-4, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
xtrain=np.asarray(final_train)
y_train=np.asarray(ytrain)

In [0]:
X_train_indices = sentences_to_indices(xtrain, word_to_index, max_len)
Y_train_oh = convert_to_one_hot(y_train, C = 2)


In [203]:
model.fit(X_train_indices, Y_train_oh,validation_split=0.2, epochs = 10, batch_size = 32, shuffle=True)

Train on 6336 samples, validate on 1584 samples
Epoch 1/10
6336/6336 [==============================] - 23s 4ms/step - loss: 0.0977 - acc: 0.9721 - val_loss: 0.3569 - val_acc: 0.8838
Epoch 2/10
6336/6336 [==============================] - 23s 4ms/step - loss: 0.0848 - acc: 0.9765 - val_loss: 0.4664 - val_acc: 0.8883
Epoch 3/10
6336/6336 [==============================] - 23s 4ms/step - loss: 0.0856 - acc: 0.9762 - val_loss: 0.4485 - val_acc: 0.8920
Epoch 4/10
6336/6336 [==============================] - 23s 4ms/step - loss: 0.0862 - acc: 0.9766 - val_loss: 0.4535 - val_acc: 0.8794
Epoch 5/10
6336/6336 [==============================] - 23s 4ms/step - loss: 0.1017 - acc: 0.9703 - val_loss: 0.3860 - val_acc: 0.8908
Epoch 6/10
6336/6336 [==============================] - 23s 4ms/step - loss: 0.0921 - acc: 0.9744 - val_loss: 0.4296 - val_acc: 0.8895
Epoch 7/10
6336/6336 [==============================] - 23s 4ms/step - loss: 0.0805 - acc: 0.9779 - val_loss: 0.4129 - val_acc: 0.8838
Epoch 8

In [0]:
test_clean=[]
for i in range(len(test)):
  new=re.sub('[^A-Za-z ]+', ' ', test['tweet'][i])
  test_clean.append(new)
  
  
  
final_test=[]
for i in range(len(test_clean)):
  final_test.append(clean_sentence(test_clean[i]))
  
  
xtest=np.asarray(final_test)


X_test_indices = sentences_to_indices(xtest, word_to_index, max_len)


y_pred=model.predict(X_test_indices)


pred=[]
for i in range(len(y_pred)):
  pred.append(y_pred[i].tolist().index(y_pred[i].max()))


In [0]:
import os
sample_sub=pd.read_csv('/content/drive/My Drive/sentiment/sample_submission_LnhVWA4.csv')
sample_sub['label']=pred
os.chdir('/content/drive/My Drive/sentiment')
sample_sub.to_csv('sub17.csv', index=False)